In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
import torch.nn as nn
import numpy as np
import random
import torch.utils.data as data
from os.path import join
from os import listdir
from PIL import Image
from torchvision.transforms import *

def is_image_file(filename):
    return any(filename.endswith(extension) for extension in [".png", ".jpg", ".jpeg",".bmp"])

def load_img(filepath, ch):
    if ch == 1:
      img = Image.open(filepath)#.convert('RGB')
    elif ch == 3:
      img = Image.open(filepath).convert('RGB')
    return img

class DatasetFromFolder(data.Dataset):
    def __init__(self, data_dir, patch_size, sigma, ch):
        super(DatasetFromFolder, self).__init__()
        self.image_filenames = [join(data_dir, data) for data in listdir(data_dir) if is_image_file(data)]
        self.patch_size = patch_size
        self.sigma = sigma
        self.ch = ch

    def __getitem__(self, index):
        img_x = load_img(self.image_filenames[index], self.ch)

        img_x = transforms.ToTensor()(img_x)
        _, h, w = img_x.shape
        ih = np.random.randint(0, h - self.patch_size)
        iw = np.random.randint(0, w - self.patch_size)

        img_x = img_x[:, ih:ih + self.patch_size, iw:iw + self.patch_size]
        noise = torch.randn(img_x.size()).mul_(self.sigma/255.0)
        img_y = img_x + noise

        return img_y, img_x

    def __len__(self):
        return len(self.image_filenames)

class DatasetFromFolderEval(data.Dataset):
    def __init__(self, data_dir, sigma, ch):
        super(DatasetFromFolderEval, self).__init__()
        self.image_filenames = [join(data_dir, data) for data in listdir(data_dir) if is_image_file(data)]
        self.sigma = sigma
        self.ch = ch

    def __getitem__(self, index):
        path, file = os.path.split(self.image_filenames[index])
        img_x = load_img(self.image_filenames[index], self.ch)

        img_x = transforms.ToTensor()(img_x)
        noise = torch.randn(img_x.size()).mul_(self.sigma/255.0)
        img_y = img_x + noise

        return img_y, img_x, file

    def __len__(self):
        return len(self.image_filenames)

def get_training_set(data_dir, patch_size, sigma, ch):
    return DatasetFromFolder(data_dir, patch_size, sigma, ch)

def get_eval_set(data_dir1, sigma, ch):
    return DatasetFromFolderEval(data_dir1, sigma, ch)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.init as init

class DnCNN(nn.Module):
    def __init__(self, args, use_bnorm=True):
        super(DnCNN, self).__init__()
        padding = 1
        layers = []

        layers.append(nn.Conv2d(in_channels=args.image_channels, out_channels=args.n_channels, kernel_size=args.kernel_size, padding=padding, bias=True))
        layers.append(nn.ReLU(inplace=True))
        for _ in range(args.depth-2):
            layers.append(nn.Conv2d(in_channels=args.n_channels, out_channels=args.n_channels, kernel_size=args.kernel_size, padding=padding, bias=False))
            layers.append(nn.BatchNorm2d(args.n_channels, eps=0.0001, momentum = 0.9))
            layers.append(nn.ReLU(inplace=True))
        layers.append(nn.Conv2d(in_channels=args.n_channels, out_channels=args.image_channels, kernel_size=args.kernel_size, padding=padding, bias=False))
        self.dncnn = nn.Sequential(*layers)
        self._initialize_weights()

    def forward(self, x):
        y = x
        out = self.dncnn(x)
        return y-out

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.orthogonal_(m.weight)
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)

In [ ]:
from __future__ import print_function
import argparse
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.utils.data import DataLoader
import time

parser = argparse.ArgumentParser(description='DnCNN network for Denoising')
parser.add_argument('--batch_size', type=int, default=32, help='training batch size')
parser.add_argument('--sigma', type=int, default=25, help='noise level')
parser.add_argument('--lr', type=float, default=1e-4, help='initial learning rate')
parser.add_argument('--start_iter', type=int, default=1, help='the starting epoch count')
parser.add_argument('--nEpochs', type=int, default=100, help='# of iter at starting learning rate')
parser.add_argument('--patch_size', type=int, default=128, help='patch size')
parser.add_argument('--snapshots', type=int, default=10, help='save weight file cycle')

parser.add_argument('--pretrained', type=bool, default=False, help='Is pretrained')
parser.add_argument('--gpu_mode', type=bool, default=True, help='GPU mode')
parser.add_argument('--threads', type=int, default=1, help='threads')
parser.add_argument('--seed', type=int, default=123, help='random seed to use')
parser.add_argument('--gpus', type=int, default=1, help='GPU nums')

parser.add_argument('--model_type', default='DnCNN', help='model name')
parser.add_argument('--depth', type=int, default=17, help='depth of network')
parser.add_argument('--n_channels', type=int, default=64, help='number of feature maps')
parser.add_argument('--image_channels', type=int, default=3, help='number of channels')
parser.add_argument('--kernel_size', type=int, default=3, help='kernel size')

parser.add_argument('--data_dir', type=str, default='/content/drive/MyDrive/denoise_dataset/train', help='dataset directory') #이 경로 설정은 본인 컴퓨터대로 해야합니다.
parser.add_argument('--save_folder', type=str, default='/content/drive/MyDrive/weights/denoising/', help='weight file save location') #이 경로 설정은 본인 컴퓨터대로 해야합니다.
args = parser.parse_args('')

gpus_list = range(args.gpus)
cudnn.benchmark = True
print(args)

def train(epoch):
    epoch_loss = 0
    model.train()
    for iteration, batch in enumerate(training_data_loader):
        img_y, img_x = Variable(batch[0]), Variable(batch[1])
        if cuda:
            img_y = img_y.cuda(gpus_list[0])
            img_x = img_x.cuda(gpus_list[0])

        optimizer.zero_grad()  #gradient를 초기에 0으로 설정
        t0 = time.time()

        prediction = model(img_y)

        loss = criterion(prediction, img_x)

        t1 = time.time()
        epoch_loss += loss.data
        loss.backward()
        optimizer.step()

        print("===> Epoch[{}]({}/{}): Loss: {:.4f} || Timer: {:.4f} sec.".format(epoch, iteration, len(training_data_loader), loss.data, (t1 - t0)))

    print("===> Epoch {} Complete: Avg. Loss: {:.4f}".format(epoch, epoch_loss / len(training_data_loader)))

def print_network(net):
    num_params = 0
    for param in net.parameters():
        num_params += param.numel()
    print(net)
    print('Total number of parameters: %d' % num_params)

def checkpoint(epoch):
    model_out_path = args.save_folder+"epoch_{}.pth".format(epoch)
    torch.save(model.state_dict(), model_out_path)
    print("Checkpoint saved to {}".format(model_out_path))

if __name__ == '__main__':

    if not os.path.exists(args.save_folder):
        os.makedirs(args.save_folder)

    cuda = args.gpu_mode
    if cuda and not torch.cuda.is_available():
        raise Exception("No GPU found, please run without --cuda")

    torch.manual_seed(args.seed)
    if cuda:
        torch.cuda.manual_seed(args.seed)

    print('===> Loading datasets')            # dataset을 로드한다.
    train_set = get_training_set(args.data_dir,args.patch_size, args.sigma, args.image_channels)
    training_data_loader = DataLoader(dataset=train_set, num_workers=args.threads, batch_size=args.batch_size, shuffle=True)

    print('===> Building model ', args.model_type)
    if args.model_type == 'DnCNN':
        model = DnCNN(args)
        model = torch.nn.DataParallel(model, device_ids=gpus_list)

    criterion = nn.L1Loss()

    if args.pretrained:
        model_name = args.save_folder + "epoch_{}.pth".format(args.start_iter - 1)
        if os.path.exists(model_name):
            model.load_state_dict(torch.load(model_name, map_location=lambda storage, loc: storage))
            print('Pre-trained Denoising model is loaded.')

    if cuda:
        model = model.cuda(gpus_list[0])
        criterion = criterion.cuda(gpus_list[0])

    optimizer = optim.Adam(model.parameters(), lr=args.lr, betas=(0.9, 0.999), eps=1e-8) # gradient 조절

    for epoch in range(args.start_iter, args.nEpochs + 1):
        train(epoch)

        if (epoch) % (args.nEpochs) == 0:
            for param_group in optimizer.param_groups:
                param_group['lr'] /= 10.0
            print('Learning rate decay: lr={}'.format(optimizer.param_groups[0]['lr']))

        if (epoch) % (args.snapshots) == 0:
            checkpoint(epoch)


Namespace(batch_size=32, sigma=25, lr=0.0001, start_iter=1, nEpochs=100, patch_size=128, snapshots=10, pretrained=False, gpu_mode=True, threads=1, seed=123, gpus=1, model_type='DnCNN', depth=17, n_channels=64, image_channels=3, kernel_size=3, data_dir='/content/drive/MyDrive/denoise_dataset/train', save_folder='/content/drive/MyDrive/weights/denoising/')
===> Loading datasets
===> Building model  DnCNN


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


===> Epoch[1](0/15): Loss: 0.5064 || Timer: 1.8308 sec.
===> Epoch[1](1/15): Loss: 0.4670 || Timer: 0.0176 sec.
===> Epoch[1](2/15): Loss: 0.4346 || Timer: 0.0088 sec.
===> Epoch[1](3/15): Loss: 0.3992 || Timer: 0.0151 sec.


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


KeyboardInterrupt: 

In [ ]:
from __future__ import print_function
import argparse

import os
import torch
import torch.nn as nn
import torch.optim as optim
import time
import cv2
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from functools import reduce
from math import log10

parser = argparse.ArgumentParser(description='DnCNN network for Denoising')
parser.add_argument('--test_batch_size', type=int, default=1, help='test batch size')
parser.add_argument('--sigma', type=int, default=25, help='noise level')
parser.add_argument('--gpu_mode', type=bool, default=True, help='GPU mode')
parser.add_argument('--threads', type=int, default=1, help='threads')
parser.add_argument('--seed', type=int, default=123, help='random seed to use')
parser.add_argument('--gpus', type=int, default=1, help='GPU nums')
parser.add_argument('--model', type=str, default="/content/drive/MyDrive/weights/denoising/epoch_10.pth", help='weight file location')

parser.add_argument('--model_type', default='DnCNN', help='model name')
parser.add_argument('--depth', type=int, default=17, help='depth of network')
parser.add_argument('--n_channels', type=int, default=64, help='number of feature maps')
parser.add_argument('--image_channels', type=int, default=3, help='number of channels')
parser.add_argument('--kernel_size', type=int, default=3, help='kernel size')

parser.add_argument('--data_dir', type=str, default='/content/drive/MyDrive/denoise_dataset/test', help='dataset directory')
parser.add_argument('--save_folder', type=str, default='/content/drive/MyDrive/result/denoising/', help='result file save location')
args = parser.parse_args('')

gpus_list=range(args.gpus)
print(args)

cuda = args.gpu_mode
if cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

torch.manual_seed(args.seed)
criterion = nn.L1Loss()

print('===> Loading datasets')
test_set = get_eval_set(args.data_dir,args.sigma,args.image_channels)
testing_data_loader = DataLoader(dataset=test_set, num_workers=args.threads, batch_size=args.test_batch_size, shuffle=False)

print('===> Building model')
model = DnCNN(args)
model = torch.nn.DataParallel(model, device_ids=gpus_list)

model.load_state_dict(torch.load(args.model, map_location=lambda storage, loc: storage))
print('Pre-trained Denoising model is loaded.')

if cuda:
    model = model.cuda(gpus_list[0])
    criterion = criterion.cuda(gpus_list[0])

def calculate_psnr(img1, img2):
    img1 , img2 = img1.cpu().numpy(), img2.cpu().numpy()
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 1.0 if img1.max() <= 1.0 else 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

def eval():
    avg_L1 = 0
    L1_sq = 0
    average = 0
    variance = 0
    avg_psnr = 0

    model.eval()
    for batch in testing_data_loader:
        with torch.no_grad():
            img_y, img_x, name = Variable(batch[0]), Variable(batch[1]), batch[2]

        if cuda:
            img_y = img_y.cuda(gpus_list[0])
            img_x = img_x.cuda(gpus_list[0])

        t0 = time.time()
        with torch.no_grad():
          prediction = model(img_y)

        L1 = criterion(prediction, img_x)
        psnr = calculate_psnr(prediction, img_x)
        avg_L1 += L1
        avg_psnr += psnr

        t1 = time.time()
        print("===> Processing: %s || Timer: %.4f sec. L1 : %.4f || PSNR : %4.f" % (name[0], (t1 - t0), L1, psnr))
        prediction = prediction.squeeze(0).cpu()
        save_img(prediction, name[0])
        average = avg_L1/len(testing_data_loader)
        average_psnr = avg_psnr / len(testing_data_loader)
    print("===> Processing Done, Average L1 : %.4f || Average PSNR : %4.f" % (average, average_psnr))

def save_img(image_tensor, filename):
    save_dir = args.save_folder

    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    save_image(image_tensor, save_dir+filename)
    print("Image saved as {}".format(save_dir+filename))

if __name__ == '__main__':
    eval()

Namespace(test_batch_size=1, sigma=25, gpu_mode=True, threads=1, seed=123, gpus=1, model='/content/drive/MyDrive/weights/denoising/epoch_10.pth', model_type='DnCNN', depth=17, n_channels=64, image_channels=3, kernel_size=3, data_dir='/content/drive/MyDrive/denoise_dataset/test', save_folder='/content/drive/MyDrive/result/denoising/')
===> Loading datasets
===> Building model
Pre-trained Denoising model is loaded.
===> Processing: 0802x2.png || Timer: 0.3003 sec. L1 : 0.0781 || PSNR :   68
Image saved as /content/drive/MyDrive/result/denoising/0802x2.png
===> Processing: 0839x2.png || Timer: 0.2065 sec. L1 : 0.0782 || PSNR :   68
Image saved as /content/drive/MyDrive/result/denoising/0839x2.png
===> Processing: 0833x2.png || Timer: 0.2375 sec. L1 : 0.0784 || PSNR :   68
Image saved as /content/drive/MyDrive/result/denoising/0833x2.png
===> Processing: 0824x2.png || Timer: 0.1935 sec. L1 : 0.0782 || PSNR :   68
Image saved as /content/drive/MyDrive/result/denoising/0824x2.png
===> Proces

KeyboardInterrupt: 